# Forecasting Demo - Logging des modèles de forecasting

3 modèles de prédiction de ventes (21 jours) : model_A, model_B, model_C

In [ ]:
import mlflow
import mlflow.pyfunc
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## Définition du modèle

In [ ]:
class ForecastingModel(mlflow.pyfunc.PythonModel):
    def __init__(self, base_range):
        self.base_range = base_range
    
    def predict(self, context, model_input):
        start_date = pd.to_datetime(model_input["date"].iloc[0])
        
        dates = [start_date + timedelta(days=i) for i in range(21)]
        min_val, max_val = self.base_range
        predictions = np.random.randint(min_val, max_val, size=21)

        df = pd.DataFrame({"date": dates, "predicted_sales": predictions})
        return df.to_dict(orient="records")

## Configuration MLflow

In [ ]:
# ici changer le nom du projet avec celui que vous avez choisi
mlflow.set_tracking_uri("http://model-platform.com/registry/my-project")

## Logging des 3 modèles

In [ ]:
models_config = {
    "model_A": (100, 500),
    "model_B": (200, 800),
    "model_C": (50, 300)
}

for model_name, (min_range, max_range) in models_config.items():
    print(f"Logging {model_name}...")
    
    with mlflow.start_run(run_name=f"{model_name.lower()}_training"):
        model = ForecastingModel(base_range=(min_range, max_range))

        mlflow.log_param("sales_range_min", min_range)
        mlflow.log_param("sales_range_max", max_range)
        mlflow.log_param("forecast_horizon_days", 21)

        mlflow.log_metric("mae", np.random.uniform(20, 50))
        mlflow.log_metric("rmse", np.random.uniform(30, 70))

        mlflow.pyfunc.log_model(
            artifact_path="custom_model",
            python_model=model,
            registered_model_name=model_name
        )
        
        print(f"   {model_name} logged successfully!")

print("All models logged!")